<a href="https://colab.research.google.com/github/AkhilVinayakp/HAR_LSTM_CNN/blob/main/lstm_cnn_model_HAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%cd /content/drive/My\ Drive/datasets 
%pwd

/content/drive/My Drive/datasets


'/content/drive/My Drive/datasets'

In [3]:

df =  pd.read_csv('int_norm.csv')

In [4]:
df.shape

(1098204, 6)

In [5]:
def create_windowed_dataset(series, window_size,shift_size, batch_size,shuffle_buffer):
  dataset = tf.data.Dataset.from_tensor_slices(series)
  dataset = dataset.window(window_size , shift = shift_size, drop_remainder = True)
  dataset = dataset.flat_map(lambda window: window.batch(window_size))
  dataset = dataset.map(lambda window: ( window[:,1:], tf.one_hot(tf.cast(window[window_size -1 , 0], tf.int64), depth=6) ))
  dataset = dataset.shuffle(shuffle_buffer)
  dataset = dataset.batch(batch_size).prefetch(1)
  return dataset

In [6]:
df.drop('timestamp', axis = 1, inplace=True)

In [7]:
df.activity = df.activity.astype('category').cat.codes
df.activity = df.activity.astype('int64')

In [8]:
train = df[df['user'] < 31 ]
test = df[df['user'] > 30]

In [9]:
train.drop('user', axis = 1, inplace=True)
test.drop('user', axis = 1, inplace=True)
train.reset_index(drop = True, inplace = True)
test.reset_index(drop=True, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [10]:
print(train.shape)
print(test.shape)

(898361, 4)
(199843, 4)


In [11]:
train.head()

,activity,x-axis,y-axis,z-axis
0,5,0.058149,0.996847,-0.053996
1,5,-0.000000,0.998427,-0.056061
2,5,-0.023448,0.997247,-0.070345
3,5,0.014845,0.998642,-0.049932
4,5,0.014737,0.998072,-0.060286


In [12]:
test.head()

,activity,x-axis,y-axis,z-axis
0,1,-0.054655,0.997718,0.039652
1,1,0.405338,0.910910,0.077102
2,1,0.410782,0.911708,-0.006846
3,1,-0.032685,0.986373,0.161248
4,1,-0.083804,0.856341,0.509566


In [28]:
# creating test set
def create_test_windowed(series, window_size = 128):
  test = tf.data.Dataset.from_tensor_slices(series)
  test = test.window(window_size, drop_remainder = True)
  test = test.flat_map(lambda window: window.batch(window_size))
  test = test.map(lambda window: (window[:,1:], tf.one_hot(tf.cast(window[window_size -1 , 0], tf.int64), depth=6)))
  return test

In [29]:
test_win = create_test_windowed(test)   # test data set for evaluation

In [ ]:
for f, l in test_win:
  print(f,l)

In [18]:
test_f = np.array(test_f)
test_f.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


(1562,)

In [ ]:
test

In [31]:
train_windowed = create_windowed_dataset(train, 128, 64, 192, 128)

In [32]:
# # adding globel average pooling to the model
# codding lstm layers and cnn
model = tf.keras.models.Sequential([
  tf.keras.layers.Lambda(lambda x: x,
                      input_shape=[128, 3]),
  tf.keras.layers.LSTM(32, return_sequences=True),
  tf.keras.layers.LSTM(32, return_sequences=True),
  tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1)),
  tf.keras.layers.Conv2D(64,5,strides=(2,2)),
  tf.keras.layers.MaxPooling2D((2, 2), strides = (2,2)),
  
  tf.keras.layers.Conv2D(128,3,strides=(1,1)),  
  tf.keras.layers.GlobalAveragePooling2D(),
  # tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1)),
  # parameters ?
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dense(6, activation='softmax')
  
])

In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda (Lambda)              (None, 128, 3)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 128, 32)           4608      
_________________________________________________________________
lstm_1 (LSTM)                (None, 128, 32)           8320      
_________________________________________________________________
lambda_1 (Lambda)            (None, 128, 32, 1)        0         
_________________________________________________________________
conv2d (Conv2D)              (None, 62, 14, 64)        1664      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 7, 64)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 5, 128)        7

In [ ]:
# with out bidirection
optimizer = tf.keras.optimizers.Adam(0.001)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics = ['accuracy'])

history = model.fit(train_windowed, epochs=200, verbose=1)

Epoch 1/200
74/74 [==============================] - 38s 49ms/step - loss: 1.3885 - accuracy: 0.6010
Epoch 2/200
74/74 [==============================] - 4s 49ms/step - loss: 0.9687 - accuracy: 0.7204
Epoch 3/200
74/74 [==============================] - 4s 50ms/step - loss: 0.8965 - accuracy: 0.7478
Epoch 4/200
74/74 [==============================] - 4s 51ms/step - loss: 0.8730 - accuracy: 0.7390
Epoch 5/200
74/74 [==============================] - 4s 50ms/step - loss: 0.8259 - accuracy: 0.7567
Epoch 6/200
74/74 [==============================] - 4s 51ms/step - loss: 0.8291 - accuracy: 0.7530
Epoch 7/200
74/74 [==============================] - 4s 50ms/step - loss: 0.8071 - accuracy: 0.7557
Epoch 8/200
74/74 [==============================] - 4s 51ms/step - loss: 0.7952 - accuracy: 0.7599
Epoch 9/200
74/74 [==============================] - 4s 50ms/step - loss: 0.8075 - accuracy: 0.7508
Epoch 10/200
74/74 [==============================] - 4s 55ms/step - loss: 0.7818 - accuracy: 0.754